## PROJECT. Исследование поведения пользователей

### ЗАДАНИЕ
необходимо проверить:
1. есть ли зависимость между выбранным уровнем сложности и вероятностью оплаты;
2. различается ли временной промежуток между регистрацией и оплатой у групп пользователей с разным уровнем сложности.

 Проверку будем проводить на основе данных пользователей, которые зарегистрировались в 2018 году (с 1 января по 31 декабря 2018 года включительно).

In [431]:
# Импортируем библиотеки, которые нам потребуются
import pandas as pd

In [432]:
# Наши данные хранятся в файлах 7_4_Events.csv и purchase.csv,которые были предварительно выгружены из БД.
events_df = pd.read_csv('data/7_4_Events.csv', sep=',')
purchase_df = pd.read_csv('data/purchase.csv', sep=',')

In [433]:
# Выделим пользователей, зарегистрированных в 2018
event_2018 = (events_df.start_time>='2018-01-01') & (events_df.start_time<'2019-01-01') & (events_df.event_type=='registration')
#Сформируем список пользователей, зарегистриванных в 2018
registered = events_df[event_2018]['user_id'].to_list()
events = events_df[events_df.user_id.isin(registered)]
events.start_time = pd.to_datetime(events.start_time, format='%Y-%m-%dT%H:%M:%S')

C:\Users\Pavel\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [434]:
purchase = purchase_df[purchase_df.user_id.isin(registered)]
purchase.event_datetime = pd.to_datetime(purchase.event_datetime, format='%Y-%m-%dT%H:%M:%S')

C:\Users\Pavel\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


# АНАЛИЗ УНИКАЛЬНЫХ ПОЛЬЗОВАТЕЛЬСКИХ ПУТЕЙ

объединим датафрейм с событиями с датафреймом по оплатам. Это позволит анализировать все эти события в рамках одной структуры данных.

Добавим в датафрейм purchase столбец event_type, который будет содержать одно значение purchase. Это нужно, чтобы в объединённом датафрейме однозначно выделить события оплаты.

In [435]:
purchase['event_type'] = 'purchase'
purchase.head(5)

C:\Users\Pavel\AppData\Local\Temp/ipykernel_12332/1758393730.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchase['event_type'] = 'purchase'


,id,user_id,event_datetime,amount,event_type
1171,16845,27845,2018-01-03 18:53:43,100,purchase
1172,16846,27865,2018-01-04 14:46:10,250,purchase
1174,16848,27911,2018-01-07 08:19:12,50,purchase
1175,16849,27910,2018-01-07 12:11:34,100,purchase
1176,16850,27940,2018-01-07 13:16:41,200,purchase


У нас есть одинаковые столбцы _id_ в двух датафреймах, но смысл их несколько отличается, так как столбец _id_ в _events_ указывает на идентификатор события, а столбец _id_ в _purchase_ указывает на идентификатор оплаты. 
Поэтому применим функцию rename(), чтобы переименовать столбцы в датафреймах.

In [436]:
events = events.rename(columns={"id": "event_id"})
purchase = purchase.rename(columns={"id": "purchase_id"})

Объединим датафреймы events и purchase с помощью функции pd.concat() и запишем объединённый датафрейм в переменную total_events_df.

In [437]:
total_events_df = pd.concat([events,purchase],sort=False)

Сбросим индексы объединённого датафрейма (т.к. после объединения они дублировались и несут мало смысла) с помощью метода _reset_index()_ и отсортируем все события по возрастанию времени с помощью _sort_values()_.

In [438]:
total_events_df = total_events_df.reset_index(drop=True).sort_values('start_time')

In [439]:
# Выведем первые 10 строк объединенного датафрейма.
total_events_df.head(10)

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
0,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
1,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
2,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
3,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
4,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN
5,80313.0,registration,NaN,2018-01-01 14:08:40,NaN,27837,NaN,NaT,NaN
6,80314.0,registration,NaN,2018-01-01 14:42:58,NaN,27838,NaN,NaT,NaN
7,80315.0,tutorial_start,NaN,2018-01-01 14:54:40,31505.0,27836,NaN,NaT,NaN
8,80316.0,tutorial_start,NaN,2018-01-01 15:00:51,31506.0,27835,NaN,NaT,NaN
9,80317.0,tutorial_finish,NaN,2018-01-01 15:06:15,31506.0,27835,NaN,NaT,NaN


### Воссоздаём последовательность событий

Теперь для каждого пользователя создаем список, который будет содержать во временной последовательности все события, совершаемые данным пользователем.

Для этого воспользуемся методом groupby по столбцу _event_type_ и применим агрегирующую функцию _apply(list)_. Таким образом мы сгруппируем строки по пользователю, а затем объединим в списки содержимое столбца _event_type_ по каждому пользователю. Запишем результат в датафрейм _user_path_df_.

In [440]:
user_path_df = (
    total_events_df.groupby(["user_id"])["event_type"].apply(list).reset_index()
)
user_path_df.head(10)

,user_id,event_type
0,27832,[registration]
1,27833,"[registration, tutorial_start, tutorial_finish]"
2,27834,"[registration, tutorial_start, tutorial_finish]"
3,27835,"[registration, tutorial_start, tutorial_finish..."
4,27836,"[registration, tutorial_start, tutorial_start,..."
5,27837,[registration]
6,27838,[registration]
7,27839,"[registration, tutorial_start, tutorial_finish..."
8,27840,"[registration, tutorial_start, level_choice]"
9,27841,"[registration, tutorial_start, tutorial_finish]"


## Проверка аналитической гипотезы

Определим, существует ли различие в частоте и средней величине оплат между тремя группами пользователей:

* пользователи, которые выбрали уровень сложнсти _easy_;
* пользователи, которые выбрали уровень сложнсти _medium_;
* пользователи, которые выбрали уровень сложнсти _hard_.

Посмотрим какие уровни сложности имеются в датафрейме events, в event_type

In [441]:
events['selected_level'].unique()

array([nan, 'medium', 'hard', 'easy'], dtype=object)

In [442]:
paying_users_count = purchase['user_id'].nunique()
display(paying_users_count)
#общее число пользователей с событием "purchase" 

1600

### Выделим группы пользователей

In [443]:
# Выделим группу пользователей, которые выбрали уровень сложности "easy" и определим их количество. 
# Запишем группу пользователей в переменную "easy_level_users", а в переменную "easy_level_users_count" - их количество.

print(events[events['selected_level'] == 'easy']['user_id'].nunique())
easy_level_users = events[events['selected_level'] == 'easy']['user_id']
easy_level_users_count = events[events['selected_level'] == 'easy']['user_id'].nunique()

2448


In [444]:
# Выделим группу пользователей, которые выбрали уровень сложности "medium" и определим их количество. 
# Запишем группу пользователей в переменную "medium_level_users", а в переменную "medium_level_users_count" - их количество.

print(events[events['selected_level'] == 'medium']['user_id'].nunique())
medium_level_users = events[events['selected_level'] == 'medium']['user_id']
medium_level_users_count = events[events['selected_level'] == 'medium']['user_id'].nunique()

4645


In [445]:
# Выделим группу пользователей, которые выбрали уровень сложности "hard" и определим их количество. 
# Запишем группу пользователей в переменную "hard_level_users", а в переменную "hard_level_users_count" - их количество.

print(events[events['selected_level'] == 'hard']['user_id'].nunique())
hard_level_users = events[events['selected_level'] == 'hard']['user_id']
hard_level_users_count = events[events['selected_level'] == 'hard']['user_id'].nunique()

1249


In [446]:
# Выделим группу пользователей, которые оплатили тренировки и выбрали уровень сложности "easy" 
# Запишем группу пользователей в отдельный датафрейм "purchase_df_easy", а в переменную "purchase_easy_users_count" - их количество.

purchase_df_easy = purchase[purchase['user_id'].isin(easy_level_users)]
print(purchase_df_easy.user_id.nunique())
purchase_easy_users_count = purchase_df_easy.user_id.nunique()

189


In [447]:
# Выделим группу пользователей, которые оплатили тренировки и выбрали уровень сложности "medium" 
# Запишем группу пользователей в отдельный датафрейм "purchase_df_medium", а в переменную "purchase_medium_users_count" - их количество.

purchase_df_medium = purchase[purchase['user_id'].isin(medium_level_users)]
print(purchase_df_medium.user_id.nunique())
purchase_medium_users_count = purchase_df_medium.user_id.nunique()

969


In [448]:
# Выделим группу пользователей, которые оплатили тренировки и выбрали уровень сложности "hard" 
# Запишем группу пользователей в отдельный датафрейм "purchase_df_hard", а в переменную "purchase_hard_users_count" - их количество.

purchase_df_hard = purchase[purchase['user_id'].isin(hard_level_users)]
print(purchase_df_hard.user_id.nunique())
purchase_hard_users_count = purchase_df_hard.user_id.nunique()

442


### Рассчитаем для каждой группы пользователей процент оплат

In [449]:
# Оценим процент пользователей, выбравших уровень сложности "easy" от числа пользователей, которые купили тренировки. 
# Запишем результат в переменную percent_easy_level:

percent_easy_level = purchase_easy_users_count / easy_level_users_count
print('Процент пользователей, которые оплатили тренировки (от числа пользователей, который выбрали уровень тренировок "easy"): {:.2%}'.format(percent_easy_level))

Процент пользователей, которые оплатили тренировки (от числа пользователей, который выбрали уровень тренировок "easy"): 7.72%


In [450]:
# Оценим процент пользователей, выбравших уровень сложности "medium" от числа пользователей, которые купили тренировки. 
# Запишем результат в переменную percent_medium_level:

percent_medium_level = purchase_medium_users_count / medium_level_users_count
print('Процент пользователей, которые оплатили тренировки (от числа пользователей, который выбрали уровень тренировок "medium"): {:.2%}'.format(percent_medium_level))

Процент пользователей, которые оплатили тренировки (от числа пользователей, который выбрали уровень тренировок "medium"): 20.86%


In [451]:
# Оценим процент пользователей, выбравших уровень сложности "hard" от числа пользователей, которые купили тренировки. 
# Запишем результат в переменную percent_hard_level:

percent_hard_level = purchase_hard_users_count / hard_level_users_count
print('Процент пользователей, которые оплатили тренировки (от числа пользователей, который выбрали уровень тренировок "hard"):{:.2%}'.format(percent_hard_level))

Процент пользователей, которые оплатили тренировки (от числа пользователей, который выбрали уровень тренировок "hard"):35.39%


## АНАЛИЗ ВРЕМЕННЫХ ПРОМЕЖУТКОВ

### Для каждой группы создадим датафреймы

In [452]:
# Создадим датафрейм "easy_level_choice_df", в котором будут только пользователи, выбравшие уровень сложности "easy". 
# Оставим в новом датафрейме только необходимые нам столбцы 'user_id', 'start_time', а также переименуем 
# столбец 'start_time' в столбец 'easy_choice_time'. 
# В заключении проверим, что на каждого уникального пользователя приходится только одно событие выбора уровня сложности. 

easy_level_choice_df = total_events_df[total_events_df['selected_level'] == 'easy']
easy_level_choice_df = easy_level_choice_df[['user_id', 'start_time']].rename(columns={'start_time':'easy_choice_time'})
easy_level_choice_df['user_id'].value_counts().mean()

1.0

In [453]:
# Подкорректируем ранее созданный датафрейм "purchase_df_easy": оставим в датафрейме только необходимые нам 
# столбцы 'user_id', 'event_datetime ', а также переименуем столбец 'event_datetime' в столбец 'easy_pay_time'.
# В заключении проверим, что на каждого уникального пользователя приходится только одно событие оплаты.

purchase_df_easy = purchase_df_easy[['user_id','event_datetime']].rename(columns={'event_datetime':'easy_pay_time'})
purchase_df_easy['user_id'].value_counts().mean()
purchase_df_easy.head(10)

,user_id,easy_pay_time
1180,27884,2018-01-08 19:37:34
1191,28182,2018-01-12 02:46:01
1193,28207,2018-01-12 21:00:24
1199,28090,2018-01-15 23:42:55
1206,28378,2018-01-18 02:11:41
1210,28247,2018-01-18 18:32:05
1218,28254,2018-01-19 22:08:40
1221,28528,2018-01-20 10:23:41
1243,28546,2018-01-23 22:55:27
1264,28984,2018-01-26 21:33:53


In [454]:
# Создадим датафрейм "medium_level_choice_df", в котором будут только пользователи, выбравшие уровень сложности "medium". 
# Оставим в новом датафрейме только необходимые нам столбцы 'user_id', 'start_time', а также переименуем 
# столбец 'start_time' в столбец 'medium_choice_time'. 
# В заключении проверим, что на каждого уникального пользователя приходится только одно событие выбора уровня сложности. 

medium_level_choice_df = total_events_df[total_events_df['selected_level'] == 'medium']
medium_level_choice_df = medium_level_choice_df[['user_id','start_time']].rename(columns={'start_time':'medium_choice_time'})
medium_level_choice_df['user_id'].value_counts().mean()

1.0

In [455]:
# Подкорректируем ранее созданный датафрейм "purchase_df_medium": оставим в датафрейме только необходимые нам 
# столбцы 'user_id', 'event_datetime', а также переименуем столбец 'event_datetime' в столбец 'medium_pay_time'.
# В заключении проверим, что на каждого уникального пользователя приходится только одно событие оплаты.

purchase_df_medium = purchase_df_medium[['user_id','event_datetime']].rename(columns={'event_datetime':'medium_pay_time'})
purchase_df_medium['user_id'].value_counts().mean()

1.0

In [456]:
# Создадим датафрейм "hard_level_choice_df", в котором будут только пользователи, выбравшие уровень сложности "hard". 
# Оставим в новом датафрейме только необходимые нам столбцы 'user_id', 'start_time', а также переименуем 
# столбец 'start_time' в столбец 'hard_choice_time'. 
# В заключении проверим, что на каждого уникального пользователя приходится только одно событие выбора уровня сложности. 

hard_level_choice_df = total_events_df[total_events_df['selected_level'] == 'hard']
hard_level_choice_df = hard_level_choice_df[['user_id','start_time']].rename(columns={'start_time':'hard_choice_time'})
hard_level_choice_df['user_id'].value_counts().mean()

1.0

In [457]:
# Подкорректируем ранее созданный датафрейм "purchase_df_hard": оставим в датафрейме только необходимые нам 
# столбцы 'user_id', 'start_time', а также переименуем столбец 'start_time' в столбец 'hard_pay_time'.
# В заключении проверим, что на каждого уникального пользователя приходится только одно событие оплаты.

purchase_df_hard = purchase_df_hard[['user_id','event_datetime']].rename(columns={'event_datetime':'hard_pay_time'})
purchase_df_hard['user_id'].value_counts().mean()

1.0

Объединим датафреймы и посчитаем разницу во времени между событиями

In [458]:
# Объединим датафрейм с пользователями, которые выбрали уровень сложности "easy" с датафреймом содержащим информацию об оплатах.
easy_level_pay_users_df = easy_level_choice_df.merge(
    purchase_df_easy, on='user_id' ,how ='inner'
    )
easy_level_pay_users_df.head(5)

,user_id,easy_choice_time,easy_pay_time
0,27884,2018-01-04 16:18:39,2018-01-08 19:37:34
1,28090,2018-01-09 21:34:23,2018-01-15 23:42:55
2,28182,2018-01-11 18:44:45,2018-01-12 02:46:01
3,28207,2018-01-11 21:10:51,2018-01-12 21:00:24
4,28254,2018-01-12 16:48:24,2018-01-19 22:08:40


In [459]:
# Создадим в датафрейме новый столбец ('timedelta') в который запишем разницу во времени между первой оплатой 
# и выбором уровня сложности. После этого посчитаем среднее значение времени.
easy_level_pay_users_df['timedelta'] = (
    easy_level_pay_users_df['easy_pay_time'] - easy_level_pay_users_df['easy_choice_time']
)
print(easy_level_pay_users_df['timedelta'].mean())

3 days 14:58:52.941798941


In [460]:
# Объединим датафрейм с пользователями, которые выбрали уровень сложности "medium" с датафреймом содержащим информацию об оплатах.
medium_level_pay_users_df = medium_level_choice_df.merge(
    purchase_df_medium, on='user_id', how='inner'
    )
medium_level_pay_users_df.head(5)

,user_id,medium_choice_time,medium_pay_time
0,27973,2018-01-07 05:29:30,2018-01-13 21:50:00
1,27981,2018-01-07 10:46:14,2018-01-07 23:20:25
2,28010,2018-01-08 00:00:52,2018-01-10 05:32:47
3,28020,2018-01-08 14:47:35,2018-01-11 21:43:03
4,28033,2018-01-08 17:06:39,2018-01-16 05:08:41


In [461]:
# Создадим в датафрейме новый столбец ('timedelta') в который запишем разницу во времени между первой оплатой 
# и выбором уровня сложности. После этого посчитаем среднее значение времени.
medium_level_pay_users_df['timedelta'] = medium_level_pay_users_df['medium_pay_time'] - medium_level_pay_users_df['medium_choice_time']
print(medium_level_pay_users_df['timedelta'].mean())

3 days 23:14:13.165118679


In [462]:
# Объединим датафрейм с пользователями, которые выбрали уровень сложности "hard" с датафреймом содержащим информацию об оплатах.
hard_level_pay_users_df = hard_level_choice_df.merge(
    purchase_df_hard, on='user_id', how='inner'
)
hard_level_pay_users_df.head(5)

,user_id,hard_choice_time,hard_pay_time
0,27845,2018-01-02 06:19:18,2018-01-03 18:53:43
1,27865,2018-01-04 05:56:32,2018-01-04 14:46:10
2,27910,2018-01-05 11:59:50,2018-01-07 12:11:34
3,27911,2018-01-05 17:39:02,2018-01-07 08:19:12
4,27940,2018-01-06 00:32:47,2018-01-07 13:16:41


In [463]:
# Создадим в датафрейме новый столбец ('timedelta') в который запишем разницу во времени между первой оплатой 
# и выбором уровня сложности. После этого посчитаем среднее значение времени.
hard_level_pay_users_df['timedelta'] = hard_level_pay_users_df['hard_pay_time'] - hard_level_pay_users_df['hard_choice_time']
print(hard_level_pay_users_df['timedelta'].mean())

3 days 07:20:41.420814479


#### Проверим, существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности

In [464]:
# Создадим датафрейм содержащий только события 'event_type' = 'registration'.  
registration_df = total_events_df[total_events_df['event_type'] == 'registration']

In [465]:
#Проверим, что в нашем датафрейме registration_df для каждого пользователя содержится только по одному событию, то есть по одной регистрации на пользователя.
registration_df['user_id'].value_counts().mean()

1.0

In [466]:
registration_df = registration_df[["user_id", "start_time"]].rename(
    columns={"start_time": "registration_time"}
)

Объеденим датафреймы registration_df и группы пользователей с разным уровнем сложности.  
Сделаем столбец timedelta, в котором посчитаем разницу между временем оплаты (*уровень сложности*_pay_time)  
и временем регистрации (registration_time):

In [467]:
#easy
merged_df_easy = registration_df.merge(
    easy_level_pay_users_df, on="user_id", how="inner"
)

merged_df_easy["timedelta"] = (
    merged_df_easy["easy_pay_time"] - merged_df_easy["registration_time"]
)
merged_df_easy["timedelta"].mean()

Timedelta('3 days 22:10:23.211640211')

In [468]:
#medium
merged_df_medium = registration_df.merge(
    medium_level_pay_users_df, on="user_id", how="inner"
)
merged_df_medium["timedelta"] = (
    merged_df_medium["medium_pay_time"] - merged_df_medium["registration_time"]
)
merged_df_medium["timedelta"].mean()

Timedelta('4 days 06:12:06.576883384')

In [469]:
#hard
merged_df_hard = registration_df.merge(
    hard_level_pay_users_df, on="user_id", how="inner"
)
merged_df_hard["timedelta"] = (
    merged_df_hard["hard_pay_time"] - merged_df_hard["registration_time"]
)
merged_df_hard["timedelta"].mean()

Timedelta('3 days 14:55:19.257918552')

# Выводы
1. Зависит ли вероятность оплаты от выбранного пользователем уровня сложности?  
По результатам проведенного анализа выявлены следующие показатели зависимости оплаты от выбранного пользователем уровня сложности:
* easy ~8%
* medium ~21%
* hard ~35%  
Таким образом, мы видим, что с изменением уровня сложности от легкого к трудному, вероятность, что пользователь совершит оплату **увеличивается**.  

2. Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности?  
Среди пользователей, выбравших уровни сложности, среднее время между *выбором уровня сложности* и *первой оплаты* составляет в среднем для:
* easy - 3 дня ~15 часов
* medium - 3 дня ~23 часа
* hard - 3 дня ~7 часов  
Как видим, быстрее всего начинают платить пользователи, выбравшие уровень сложности "hard", им требуется в среднем 3 дня и 7,5 часов.  
Самым длинный временной отрезок между событиями выбора уровня сложности и оплаты - оказался для группы пользователей, которые выбрали уровень **medium** - он составляет ~4 дня  
Разница между событиями *регистрации* и *оплаты* для разных групп пользователей с *разным уровнем сложности* составляет:
* easy - 3 дня ~22 часа
* medium - 4 дня ~6 часов
* hard - 3 дня ~15 часов  
Отвечая на основной вопрос из задания про разницу во времени выделим следующее наблюдение: самый короткий временной промежуток от регистрации до оплаты составил 3д 15ч для группы пользователей, выбравших уровень **hard**.  
